In [1]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
plt.style.use('default')
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, PCA
from sklearn.cluster import KMeans
from sklearn.manifold import Isomap, TSNE
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed, Embedding, GlobalAveragePooling1D
from tensorflow.keras.utils import plot_model
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

import requests
stopwords_list = requests.get("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").content
stopwords = set(stopwords_list.decode().splitlines()) 
import os

# Aula 18: Redes Neurais ao Longo do Tempo
**Objetivo: : ao fim desta aula, o aluno será capaz de aplicar redes neurais recorrentes para agregar informações de documentos ao longo do tempo**

In [ ]:
df = pd.read_csv('./datasets/IMDB Dataset.csv')
reviews = list(df['review'])

labels = np.array([list(df['sentiment'])]).T
ohe = OneHotEncoder()
y = ohe.fit_transform(labels).toarray()

# Exercício 1
*Objetivo: entender como sequências são representadas no Keras*

Um documento pode ser entendido como uma sequência de palavras.

1. Se representássemos nossa coleção como sequências de palavras, chegaríamos a uma matriz. Que significaria cada linha e coluna da matriz?

1. Como deveríamos lidar com o fato de que documentos podem ter mais ou menos palavras que a dimensão correspondente da matriz?

1. Se cada palavra for representada por um vetor de N dimensões, então nossa coleção passaria a ser representada por um tensor, que é essencialmente uma matriz 3D (na verdade, tensores podem ter várias dimensões, mas usaremos somente 3 neste caso). O que significa cada dimensão desse tensor?

1. No código de pré-processamento abaixo, o que fazem as funções `tokenizer.texts_to_sequences()` e `pad_sequences()`?


In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)

In [ ]:
print(sequences[0])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded = pad_sequences(sequences,maxlen=200)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded, y, test_size=0.2)   

# Exercício 2
*Objetivo: explicar uma rede neural à partir de sua representação gráfica*

Nesta aula, usaremos a camada `embedding`. A camada `embedding` relaciona cada palavra a uma posição em um espaço vetorial – como se estivéssemos multiplicando uma representação one-hot encoding das palavras por uma matriz. A diferença entre a matriz e o embedding é que a camada embedding faz essa transformação através de um dicionário cuja chave é a palavra e o conteúdo é a representação vetorial correspondente.

Veja o código da implementação `rede_neural_classificar_por_palavra` abaixo. Se precisar, use a documentação do Keras para responder:

1. O que a camada `GlobalAveragePooling1D` faz? Em que dimensão (documentos, dimensões latentes ou tempo) ela opera?
1. Esta rede é semelhante a qual outra rede que já trabalhamos nesta disciplina?
1. O que esperamos encontrar no espaço latente desta rede?

In [ ]:
def rede_neural_classificar_por_palavra(input_dims, n_dims_out):
  input_layer = Input(shape=(input_dims,))
  x = input_layer
  x = Embedding(1000, 2, name='projecao')(x)
  x = GlobalAveragePooling1D()(x)
  y = Dense(2, activation='sigmoid', name='classificador')(x)
  return Model(input_layer, y)

rede_neural = rede_neural_classificar_por_palavra(200, 2)
rede_neural.compile(optimizer='adam', loss='mse')
plot_model(rede_neural, show_shapes=True, show_layer_activations=True)

# Exercício 3
*Objetivo: aplicar uma rede neural com camada embedding e interpretar os resultados*

Execute o treino e teste da rede neural.
1. Os resultados condizem com o que você esperaria encontrar?
1. Aumente o número de dimensões do espaço latente. O desempenho da rede muda? Usando `PCA`, analise o espaço latente de mais alta dimensão. Você consegue visualizar diferenças?

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
history = rede_neural.fit(X_train, y_train, epochs=500, validation_split=0.2, callbacks=es)

In [ ]:
plt.figure(figsize=(7,2))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
y_est = rede_neural.predict(X_test)
print(classification_report(ohe.inverse_transform(y_test), ohe.inverse_transform(y_est)))

## Analisando layer de Embedding

In [ ]:
w = rede_neural.get_layer('projecao').get_weights()

In [1]:
# Visualização: onde foi parar cada palavra?
v_ = rede_neural.get_layer('projecao').get_weights()[0]

#proj = PCA(n_components=2, perplexity=5)
#v = proj.fit_transform(v_)
v = v_

plt.figure(figsize=(4,4))
plt.scatter(v[:,0], v[:,1], s=1, alpha=0.3, c='b')
for s in ["director", "actor", "bad", "good", "excellent", "plot", "worst", "terrible", "waste", "awful", "fantastic"]:
    _n = tokenizer.texts_to_sequences([[s]])[0][0]
    plt.text(v[_n,0], v[_n,1], s, ha='center')
plt.title('Projeção das palavras no espaço latente')
plt.ylabel('Componente 2')
plt.xlabel('Componente 1')
#plt.xlim([-20,20])
#plt.ylim([-20,20])
plt.show()

NameError: name 'rede_neural' is not defined

# Exercício 4
*Objetivo: entender como uma rede recorrente opera no domínio do tempo*

1. Uma rede neural recorrente aplica alguma variação da equação:
$$
y[t] = f( \[x[t], y[t-1]\])
$$

Isso significa que uma de suas entradas é a saída na interação anterior.

Desenhe um diagrama de blocos mostrando esse comportamento.

1. A rede recorrente opera no domínio do tempo. Trata-se de um tempo "discreto", ou seja, existe t=1, t=2, t=3, etc. No caso da frase "frase de teste", quantos "tempos discretos" devem existir?

1. A rede neural recorrente recebe como entradas a entrada de fato (dados) e também a saída que foi gerada na iteração anterior. Por que não é possível usar como entrada a saída da iteração atual?

1. Podemos dizer que esse tipo de rede tem algum tipo de "memória" quanto ao passado? Como isso se relaciona a modelos do tipo n-grama?

1. De acordo com o diagrama de blocos, a rede emite uma saída para cada instante de tempo. No nosso caso, isso significa uma saída para cada palavra da frase. Consulte a documentação do Keras e encontre: como o bloco `SimpleRNN` pode retornar uma única saída para cada documento, isto é, como a dimensão do tempo é eliminada?



In [ ]:
from keras.layers import SimpleRNN
def rede_neural_com_RNN(input_dims, n_dims_out):
  input_layer = Input(shape=(input_dims,))
  x = input_layer
  x = Embedding(1000, 2, name='projecao')(x)
  x = SimpleRNN(2, activation='linear')(x)
  y = Dense(2, activation='sigmoid', name='classificador')(x)
  return Model(input_layer, y)

rede_neural = rede_neural_com_RNN(200, 2)
rede_neural.compile(optimizer='adam', loss='mse')
plot_model(rede_neural, show_shapes=True, show_layer_activations=True)

# Exercício 5
*Objetivo: avaliar uma rede neural recorrente para o problema de classificação de sentimentos em avaliações de filmes*

Avalie a rede neural recorrente presente no notebook. Como ela se comporta em relação ao:

1. Desempenho nas métricas usuais
1. Propriedades do espaço latente



In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
history = rede_neural.fit(X_train, y_train, epochs=500, validation_split=0.2, callbacks=es)

In [ ]:
plt.figure(figsize=(7,2))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
y_est = rede_neural.predict(X_test)
print(classification_report(ohe.inverse_transform(y_test), ohe.inverse_transform(y_est)))

In [ ]:
# Visualização 2: onde foi parar cada palavra?
v_ = rede_neural.get_layer('projecao').get_weights()[0]

#proj = PCA(n_components=2, perplexity=5)
#v = proj.fit_transform(v_)
v = v_

plt.figure(figsize=(4,4))
plt.scatter(v[:,0], v[:,1], s=1, alpha=0.3, c='b')
for s in ["director", "actor", "bad", "good", "excellent", "plot", "worst", "terrible", "waste", "awful", "fantastic"]:
    _n = tokenizer.texts_to_sequences([[s]])[0][0]
    plt.text(v[_n,0], v[_n,1], s, ha='center')
plt.title('Projeção das palavras no espaço latente')
plt.ylabel('Componente 2')
plt.xlabel('Componente 1')
#plt.xlim([-20,20])
#plt.ylim([-20,20])
plt.show()

# Exercício 6
*Objetivo: usar embeddings pré-treinados*

Fonte: [Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/)

1. Em nosso espaço latente, criado pela propagação do gradiente no problema de classificação, o que a posição de uma palavra pode nos informar sobre ela?

1. Seria possível "emprestar" o embedding criado em um outro processo de classificação, e então treinar somente a etapa de classificação em seu problema específico?

1. Um espaço latente muito usado hoje é o GloVe. Ele foi criado por uma equipe de Stanford e tem duas propriedades muito interessantes. As duas propriedades estão discutidas no seu [website](https://nlp.stanford.edu/projects/glove/). Quais são essas propriedades e como elas se assemelham ao que já estudamos neste curso?

# Exercício 7
*Objetivo: usar e interpretar um classificador que use embeddings pré-treinados de GloVe*

Analise e execute a rede neural que usa embeddings GloVe.
1. Como ela se compara com as anteriores?
1. O que significa o parâmetro `trainable=False` da camada de embedding?
1. Como se comporta o espaço latente de GloVe em relação às palavras que usamos? Como devemos interpretar esse espaço latente?


In [ ]:
# Abrindo os embeddings GloVe
f = open("./datasets/glove.6B.100d.txt", encoding="utf8")
embeddings_index = dict()
for line in f:
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        continue
f.close()


In [ ]:
# Reconstruindo a matriz de embeddings (para carregar na camada embedding)
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [ ]:
from keras.layers import SimpleRNN
def rede_neural_com_Glove(input_dims, n_dims_out):
  input_layer = Input(shape=(input_dims,))
  x = input_layer
  x = Embedding(len(word_index) + 1, 100, name='projecao', weights=[embedding_matrix], trainable=False)(x)
  x = SimpleRNN(2, activation='linear')(x)
  y = Dense(2, activation='sigmoid', name='classificador')(x)
  return Model(input_layer, y)

rede_neural = rede_neural_com_Glove(200, 2)
rede_neural.compile(optimizer='adam', loss='mse')
plot_model(rede_neural, show_shapes=True, show_layer_activations=True)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
history = rede_neural.fit(X_train, y_train, epochs=500, validation_split=0.2, callbacks=es)

In [ ]:
plt.figure(figsize=(7,2))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
y_est = rede_neural.predict(X_test)
print(classification_report(ohe.inverse_transform(y_test), ohe.inverse_transform(y_est)))

In [ ]:
# Visualização 2: onde foi parar cada palavra?
v_ = rede_neural.get_layer('projecao').get_weights()[0]

proj = PCA(n_components=2)
v = proj.fit_transform(v_)


plt.figure(figsize=(4,4))
plt.scatter(v[:,0], v[:,1], s=1, alpha=0.01, c='b')
for s in ["director", "actor", "bad", "good", "excellent", "plot", "worst", "terrible", "waste", "awful", "fantastic"]:
    _n = tokenizer.texts_to_sequences([[s]])[0][0]
    plt.text(v[_n,0], v[_n,1], s, ha='center')
plt.title('Projeção das palavras no espaço latente')
plt.ylabel('Componente 2')
plt.xlabel('Componente 1')
#plt.xlim([-20,20])
#plt.ylim([-20,20])
plt.show()

# Exercício 8
*Objetivo: avaliar se a rede recorrente está usando informações de tempo*

Você deve ter observado que usar redes recorrentes é bem mais demorado que usar somente `GlobalAveragePooling1D` ou mesmo que usar as matrizes de DF que usamos na aula anterior. Essa desvantagem pode se compensar: é comum encontrarmos argumentos que dizem que as redes recorrentes acabam conseguindo modelar um pouco da relação temporal entre as palavras. Por outro lado, é possível (ainda não verificamos) que a rede neural recorrente tenha convergido para um estado que somente replica a camada `GlobalAveragePooling1D`.

Proponha e execute um experimento capaz de verificar se o desempenho da rede neural recorrente, ao menos neste dataset, se deve ao modelamento da relação temporal entre palavras.
